In [387]:
# general
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from collections import Counter
import collections
from pandas import DataFrame
from scipy.cluster.hierarchy import linkage, dendrogram

# sk learn
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.feature_extraction import text 
from sklearn.metrics.pairwise import linear_kernel
import matplotlib.pyplot as plt 

In [19]:
# traveler profile
data3 = pd.read_excel('data/users_full_7034.xlsx')
# traveler and reviews
data4 = pd.read_excel('data/reviews_32618_for_1098_users_with_location.xlsx')

In [20]:
null_age = data3.ageRange.isnull()
null_gender = data3.gender.isnull()
null_style = data3.travelStyle.isnull()

In [21]:
filtered_data3 = data3[data3.totalPoints > 1000][~null_age][~null_gender][~null_style]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [35]:
filtered_data3 = filtered_data3[['username']]

In [36]:
filtered_data4 = data4[['id', 'username', 'type', 'title', 'text', 'rating', 'taObjectCity']]


In [37]:
attraction_only = filtered_data4.type == 'Attractions'
filtered_data4 = filtered_data4[attraction_only]

In [38]:
data3_merge_4 = pd.merge(filtered_data4, filtered_data3, on=['username'])


In [1023]:
# cities with more than 4 reviews
popular_city = []
for item, value in Counter(data3_merge_4.taObjectCity).items():
    if value > 7:
        popular_city.append(item)
print(len(popular_city))

59


In [1024]:
final_df = data3_merge_4[data3_merge_4.taObjectCity.isin(popular_city)]


In [1025]:
final_df.head()

,id,username,type,title,text,rating,taObjectCity
2,362,19Cam,Attractions,See it before its gone,The box is only a temporary exhibit while cons...,4,Berlin
3,363,19Cam,Attractions,Worth a look for the architecture alone,"A large collection, presented in a stunning bu...",3,Berlin
4,378,19Cam,Attractions,Fantastic,"After reading all the hype about this show, I ...",5,London
5,414,19Cam,Attractions,Tranquility in a busy city,Sited within the main botanical gardens area o...,4,Kuala Lumpur
18,778,1oldseagull,Attractions,THIS SHOULD BE A FIRST STOP,"First thing, this visiter center was not easy ...",5,Chattanooga


In [1026]:
final_df.tail(5)

,id,username,type,title,text,rating,taObjectCity
2617,32510,FamilyTravelJunkkie,Attractions,If you travel at the right time there are no c...,Disneyland is great and so is California Adven...,4,Anaheim
2622,32516,FamilyTravelJunkkie,Attractions,If you are a cricket fan,If you are a cricket fan Kensington Oval is de...,4,Bridgetown
2632,32580,Farnborough,Attractions,A bit of a joke really,This is not actually the original or genuine C...,1,Berlin
2635,32612,Farnborough,Attractions,Not worth visiting,"As cat lovers, when we saw this mentioned in o...",1,Amsterdam
2636,32614,Farnborough,Attractions,Didn&acute;t live up to expectations,This didnвЂ™t really live up to expectations f...,2,Amsterdam


In [1027]:
final_df.shape

(991, 7)

# clustering functions

In [1028]:
cluster_input_df = final_df.copy()
cluster_input_df = cluster_input_df[['title', 'text', 'taObjectCity']]

In [1029]:
def combine(lst):
    return lst.

SyntaxError: invalid syntax (<ipython-input-1029-4ae4a2a19120>, line 2)

In [1030]:
cluster_input_df.head()

,title,text,taObjectCity
2,See it before its gone,The box is only a temporary exhibit while cons...,Berlin
3,Worth a look for the architecture alone,"A large collection, presented in a stunning bu...",Berlin
4,Fantastic,"After reading all the hype about this show, I ...",London
5,Tranquility in a busy city,Sited within the main botanical gardens area o...,Kuala Lumpur
18,THIS SHOULD BE A FIRST STOP,"First thing, this visiter center was not easy ...",Chattanooga


In [1031]:
final_df.head()

,id,username,type,title,text,rating,taObjectCity
2,362,19Cam,Attractions,See it before its gone,The box is only a temporary exhibit while cons...,4,Berlin
3,363,19Cam,Attractions,Worth a look for the architecture alone,"A large collection, presented in a stunning bu...",3,Berlin
4,378,19Cam,Attractions,Fantastic,"After reading all the hype about this show, I ...",5,London
5,414,19Cam,Attractions,Tranquility in a busy city,Sited within the main botanical gardens area o...,4,Kuala Lumpur
18,778,1oldseagull,Attractions,THIS SHOULD BE A FIRST STOP,"First thing, this visiter center was not easy ...",5,Chattanooga


In [1032]:
span_mask1 = (final_df.username == 'AnaS1')
span_mask2 = (final_df.username == 'DaniLK')
span_mask3 = (final_df.username == 'Aprile_24')
non_city_mask = (final_df.taObjectCity == 'California')


In [1033]:
final_df = final_df[~span_mask1][~span_mask2][~non_city_mask][~span_mask3]

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [1034]:
final_df.shape

(980, 7)

In [1035]:
# print(final_df.text.values[0])
# print(final_df.text.values[1])

In [1036]:
df_title_comb = final_df.groupby(['taObjectCity']).apply(lambda x: ' '.join(x.title)).reset_index()
df_text_comb = final_df.groupby(['taObjectCity']).apply(lambda x: ' '.join(x.text)).reset_index()

In [1037]:
cluster_input_df = df_title_comb.merge(df_text_comb, on= 'taObjectCity')

In [1038]:
cluster_input_df.head()

,taObjectCity,0_x,0_y
0,Amsterdam,An interesting Story from start to finish A Mu...,We found Van Gogh&acute;s story so interesting...
1,Anaheim,Cars land a great addition Easy shopping an ol...,Disney has taken time to build the new cars la...
2,Bangkok,Impressive indeed! Must do also :) Nice one - ...,Every Bangkok visit should include the Grand p...
3,Barcelona,OK More than a club Just wow! Cosmo Scooter - ...,"The gardens are very pleasant, but nothing out..."
4,Bergen,A Great Hike! Touristy and Pricey A Must-Do......,A hike that will leave you tired but smiling. ...


In [1039]:
cluster_input_df.columns = ['taObjectCity','title','text']
cluster_input_df.head()

,taObjectCity,title,text
0,Amsterdam,An interesting Story from start to finish A Mu...,We found Van Gogh&acute;s story so interesting...
1,Anaheim,Cars land a great addition Easy shopping an ol...,Disney has taken time to build the new cars la...
2,Bangkok,Impressive indeed! Must do also :) Nice one - ...,Every Bangkok visit should include the Grand p...
3,Barcelona,OK More than a club Just wow! Cosmo Scooter - ...,"The gardens are very pleasant, but nothing out..."
4,Bergen,A Great Hike! Touristy and Pricey A Must-Do......,A hike that will leave you tired but smiling. ...


In [1040]:
cluster_input_df.set_index(['taObjectCity'], drop=True, inplace=True)

In [1041]:
cluster_input_df.head()

,title,text
taObjectCity,,
Amsterdam,An interesting Story from start to finish A Mu...,We found Van Gogh&acute;s story so interesting...
Anaheim,Cars land a great addition Easy shopping an ol...,Disney has taken time to build the new cars la...
Bangkok,Impressive indeed! Must do also :) Nice one - ...,Every Bangkok visit should include the Grand p...
Barcelona,OK More than a club Just wow! Cosmo Scooter - ...,"The gardens are very pleasant, but nothing out..."
Bergen,A Great Hike! Touristy and Pricey A Must-Do......,A hike that will leave you tired but smiling. ...


In [1142]:


def cluster_texts(texts, clusters=3):
    """ 
    Transform texts to Tf-Idf coordinates and cluster texts using K-Means 
    """
    my_additional_stop_words = ['acute', 'good', 'great', 'really', 'just', 'nice', 'like', 'day']
    stop_words = text.ENGLISH_STOP_WORDS.union(my_additional_stop_words)
    
    
    vectorizer = TfidfVectorizer(stop_words= stop_words,
                                 max_features = 800,
                                 lowercase=True)
 
    tfidf_model = vectorizer.fit_transform(texts)
    vectors = tfidf_model.toarray()
    cols = vectorizer.get_feature_names()

    return (vectors, cols)


def cluster(vectors, cols, texts):
    """
    Cluster vecotirzed reviews and create k books a data frame relating the
    k label to the book id
    """
    kmeans = KMeans(n_clusters=5, random_state=347837898).fit(vectors)
    k_books = pd.DataFrame(list(zip(list(kmeans.labels_),
                                list(texts.index))),
                                columns=['cluster_k', 'city_index'])
    
    ''' added code to print centriod vocab - Print the top n words from all centroids vocab
    '''
    n = 20
    centroids = kmeans.cluster_centers_
    for ind, c in enumerate(centroids):
        print(ind)
        indices = c.argsort()[-1:-n-1:-1]
        print([cols[i] for i in indices])
        print("=="*20)
    
    return k_books

In [1143]:
reviews = cluster_input_df.title + ' ' + cluster_input_df.text
#reviews = final_df.title + ' ' + final_df.text
content = [i for i in reviews]


In [1144]:
vector, cols = cluster_texts(content, 5)
cluster_df = cluster(vector, cols, reviews)

0
['sydney', 'beach', 'tour', 'worth', 'trip', 'aquarium', 'displays', 'west', 'time', 'value', 'harbour', 'gallery', 'fantastic', 'love', 'walk', 'opportunities', 'boat', 'air', 'visit', 'crew']
1
['train', 'hk', 'dubai', 'fun', 'free', 'view', 'main', 'centre', 'gate', 'place', 'experience', 'busy', 'need', 'minutes', 'central', 'station', 'amazing', 'food', 'city', 'stadium']
2
['falls', 'trail', 'hike', 'memorial', 'valley', 'yellowstone', 'water', 'road', 'area', 'views', 'right', 'walk', 'hiking', 'beach', 'easy', 'view', 'grove', 'ride', 'worth', 'visit']
3
['tour', 'time', 'museum', 'visit', 'place', 'worth', 'fun', 'park', 'history', 'lot', 'experience', 'guide', 'interesting', 'little', 'did', 'way', 'people', 'building', 'small', 'took']
4
['tour', 'visit', 'place', 'beautiful', 'inside', 'worth', 'building', 'city', 'lovely', 'walk', 'museum', 'impressive', 'outside', 'interesting', 'time', 'excellent', 'old', 'art', 'view', 'barcelona']


In [1149]:
cluster2_mask = (cluster_df['cluster_k'] ==2)
# random state - 347837898


In [1150]:
cluster_df[cluster2_mask]

,cluster_k,city_index
4,2,Bergen
15,2,Honolulu
34,2,Niagara Falls
57,2,Yellowstone National Park
58,2,Yosemite National Park
